In [21]:
import os
import numpy
import csv

In [54]:
with open('out.csv') as csvfile:
    data = csv.reader(csvfile, delimiter = ',')
    events = []
    times = []
    for row in data:
        event = row[0]
        events.append(event)
        time = row[1]
        times.append(time)

In [73]:
for i in range(10):
    print(TOtimes[i])

 11.0469
 203.8456
 400.2447
 606.6422
 779.6449
 945.4428
 1119.8423
 1391.2433
 1685.6392
 2562.0398


In [97]:
TargetOnset = []
Saccade = []
for i, x in enumerate(events):
    if x == '2651':
        TargetOnset.append(i)
    if x == '888':
        Saccade.append(i)

In [98]:
len(TargetOnset)

1122

In [68]:
len(Saccade)

1097

In [71]:
TOtimes = []
for items in TargetOnset:
    TOtimes.append(times[items])

In [108]:
Trials = []
for i, x in enumerate(TargetOnset):
    if i < len(TargetOnset)-1:
        a = TargetOnset[i]
        b = TargetOnset[i+1]-1
        Trials.append(events[a:b])
    else:
        a = TargetOnset[i]
        Trials.append(events[a:])

In [109]:
print(Trials[1121])

['2651', '0', '2300', '0', '2810', '0', '888', '0', '2600', '0', '2001', '777', '0', '2727', '1667', '2998', '3999', '4001', '0', '4052', '4060', '4101', '4152', '4201', '0', '4258', '4300', '4350', '4400', '5090', '0', '5635', '3800', '4650', '4664', '4672', '0', '4700', '4800', '4917', '6000', '3007', '0', '3200', '5000', '11000', '3100', '3600', '0', '3100', '3030', '4500', '4000', '4500', '0', '4000', '4690', '4790', '2999', '0']
